# SCRAP DATA

In [ ]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os
import random
import numpy as np
import time



# site to scrap
source = 'bikez'

# url to scrap
url = 'https://bikez.com/years/index.php'
response = requests.get(url)
    
soup = BeautifulSoup(response.content, "html.parser")

count_year = 0

for year in soup.find('table', class_="zebra").find_all('td'):
    try:
        url_year = "https://bikez.com" + year.find("a").get('href')[2:]
        bike_year = year.find("a").text
        print(bike_year)

        response_year = requests.get(url_year)
        soup_year = BeautifulSoup(response_year.content, "html.parser")

        link_list = [link for link in soup_year.find('table', class_="zebra").find_all('a')]
        link_result=[]

        for link in link_list:
            try:
                link_result.append(link.get('href')[2:])
            except:
                pass

        bike_list = ["https://bikez.com" + link  for link in link_result if 'motorcycles'in link]
        # remove duplicates 
        bike_list = list(dict.fromkeys(bike_list))

        bike_count = 0

        for url_bike in bike_list:
            print(url_bike)
            response_bike = requests.get(url_bike)
            soup_bike = BeautifulSoup(response_bike.content, "html.parser")

            features = ['Model:', 
                        'Year:',
                        'Category:', 
                        'Engine type:',
                        'Displacement:',
                        'Power:',
                        'Torque:',
                        'Compression:',
                        'Bore x stroke',
                        'Fuel system',
                        'Cooling system:',
                        'Shaft drive (cardan)',
                        'Wheels:',
                        'Dry weight:',
                        'Power/weight ratio:']

            features_dict = {
                'Model:':[np.nan], 
                'Year:':[np.nan],
                'Category:':[np.nan], 
                'Engine type:':[np.nan],
                'Displacement:':[np.nan],
                'Power:':[np.nan],
                'Torque:':[np.nan],
                'Compression:':[np.nan],
                'Bore x stroke':[np.nan],
                'Fuel system':[np.nan],
                'Cooling system:':[np.nan],
                'Shaft drive (cardan)':[np.nan],
                'Wheels:':[np.nan],
                'Dry weight:':[np.nan],
                'Power/weight ratio:':[np.nan]}

            rows = soup_bike.find_all('td')
            data = []

            for row in rows:
                data.append(row.text)

            for feature in features:
                try:
                    position = data.index(feature)+1
                    features_dict[feature]= [data[position]]
                except:
                    pass

            df = pd.DataFrame(features_dict)
            df.to_csv(f"bike/{features_dict['Model:']}-{features_dict['Year:']}.csv")
            print(bike_count)
            bike_count +=1

        count_year += 1
    except:
        pass

# BUILD DATAFRAME AND CLEAN

In [1]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os
import random
import numpy as np
import time

history = pd.DataFrame(
    data = {'Unnamed: 0':[], 'Model:':[], 'Year:':[], 'Category:':[], 'Engine type:':[],
       'Displacement:':[], 'Power:':[], 'Torque:':[], 'Compression:':[], 'Bore x stroke':[],
       'Fuel system':[], 'Cooling system:':[], 'Shaft drive (cardan)':[], 'Wheels:':[],
       'Dry weight:':[], 'Power/weight ratio:':[]})

for filename in [file for file in os.listdir('bike') if file.endswith(".csv")]:
    df = pd.read_csv(f"bike/{filename}")
    history = history.append(df)

history.to_csv("bikez_raw.csv", index=False)

In [16]:
df = pd.read_csv("bikez_raw.csv")
new_columns_name = [column.lower().replace(':','') for column in df.columns]
df.columns=new_columns_name
df.drop(columns=['unnamed 0'], inplace=True)

df['power/weight ratio'] = df['power/weight ratio'].str.replace(" HP/kg", "")

def parser(to_clean, delimiter):
    try:
        if type(to_clean)==float:
            return np.nan
        cleaned = to_clean.split(f' {delimiter}')[0].replace(",","")
        return float(cleaned)
    except:
        return np.nan

df.displacement = df.apply(lambda x: parser(x['displacement'],'ccm'), axis=1)
df.power = df.apply(lambda x: parser(x['power'],'HP'), axis=1)
df.torque = df.apply(lambda x: parser(x['torque'],'Nm'), axis=1)
df['dry weight'] = df.apply(lambda x: parser(x['dry weight'],'kg'), axis=1)

df.drop(columns=['bore x stroke', 'fuel system','shaft drive (cardan)','wheels'], inplace=True)

models = pd.read_csv('bikez_models.csv')

models.drop(columns=['Unnamed: 0'], inplace=True)

brand_list = models.brand.to_list()

model_list = models.model.to_list()

def return_model(model_):
    for model in model_list:
        if str(model) in str(model_):
            return str(model)
    return np.nan
    

def clean_model(model):
    for brand in brand_list:
        if str(brand) in str(model):
            return brand
    return np.nan

def clean_model_2(model, brand):
    return str(model).replace(f"{brand} ",'')

df['model_'] = df.model.apply(return_model)

df['brand'] = df.model.apply(clean_model)

df.model = df.apply(lambda x: clean_model_2(x['model'], x['brand']), axis=1)
df.model_ = df.apply(lambda x: clean_model_2(x['model_'], x['brand']), axis=1)

df.dropna(subset=['year'], inplace=True)

df.year = df.year.astype(int)

new_names = ['model_submodel', 'year', 'category', 'engine_type', 'displacement', 'power',
       'torque', 'compression', 'cooling_system', 'dry_weight',
       'power/weight_ratio', 'model', 'brand']

df.columns=new_names

def reverse_sentence(sentence):
    return " ".join(sentence.split()[::-1])

df['model_submodel_inv'] = df['model_submodel'].apply(reverse_sentence)
df['model_inv'] = df['model'].apply(reverse_sentence)

#df['raw_brand'] = df.brand.str.lower()
#df['raw_model'] = df.model.str.lower()
#df['raw_model_submodel'] = df.model_submodel.str.lower()
#df['raw_brand_model'] = df.apply(lambda x: str(x['brand']).lower() + " " + str(x['model']).lower() ,axis=1)
#df['raw_brand_submodel'] = df.apply(lambda x: str(x['brand']).lower() + " " + str(x['model_submodel']).lower(),axis=1)

# brand and model cleaning
df.model_submodel_inv = df.model_submodel_inv.str.lower()
df.model_submodel = df.model_submodel.str.lower()
df.model = df.model.str.lower()
df.model_inv = df.model_inv.str.lower()
df.brand = df.brand.str.lower()
df.model_submodel_inv = df.model_submodel_inv.str.strip()
df.model_submodel = df.model_submodel.str.strip()
df.model = df.model.str.strip()
df.model_inv = df.model_inv.str.strip()
df.brand = df.brand.str.strip()
df.model_submodel_inv = df.model_submodel_inv.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.model_submodel = df.model_submodel.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.model = df.model.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.model_inv = df.model_inv.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.brand = df.brand.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")

#clean category
df.category = df.category.str.lower().str.strip().str.replace('motorcycle', '')

# engine type
def parser_engine_type(engine_type):
    try:
        return engine_type.split(',')[0]
    except:
        return np.nan

df['engine_type'] = df.apply(lambda x: parser_engine_type(x['engine_type']), axis=1)
#clean engine type
df['engine_type'] = df['engine_type'].str.lower().str.strip()

df.columns = [x+"_db" for x in df.columns]
df.rename(columns={'displacement_db': 'engine_size_db', 'type_db': 'category_db'},  inplace=True)
#'raw_brand_db', 'raw_model_db', 'raw_model_submodel_db','raw_brand_model_db', 'raw_brand_submodel_db',
df = df[['brand_db','model_db', 'model_inv_db', 'model_submodel_db','model_submodel_inv_db', 'year_db', 'category_db', 'engine_type_db',
       'engine_size_db', 'power_db', 'torque_db', 'compression_db',
       'cooling_system_db', 'dry_weight_db', 'power/weight_ratio_db']]

df['model_size_db'] = df.apply(lambda x: str(x['model_db'])+str(x['engine_size_db']).replace('.0',''),axis=1)
df['model_size_inv_db'] = df['model_size_db'].apply(reverse_sentence)

df.to_csv('bikez.csv')

<ipython-input-16-f3099db4064f>:88: FutureWarning: The default value of regex will change from True to False in a future version.
  df.model_submodel_inv = df.model_submodel_inv.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
<ipython-input-16-f3099db4064f>:89: FutureWarning: The default value of regex will change from True to False in a future version.
  df.model_submodel = df.model_submodel.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
<ipython-input-16-f3099db4064f>:90: FutureWarning: The default value of regex will change from True to False in a future version.
  df.model = df.model.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
<ipython-input-16-f3099db4064f>:91: FutureWarning: The default value of regex will change from True to False in a future version.
  df.model_inv = df.model_inv.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
<ipython-input-16-f3099db4064f>:92: FutureWarning: T

# IMPORT AND VISUALIZE

In [ ]:
data = pd.read_csv('master_model_motorcycledb.csv')
data.dropna(subset=['year_db'],inplace=True)
data.year_db = data.year_db.astype(int)
data.to_csv('master_model_motorcycledb.csv', index=False)

In [ ]:
data.head()

In [ ]:
data[data.raw_model.str.contains('trom')]